# Jupyter Notebook to clean install all the contracts


### Necessary packages 

In [56]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


# Set Up Environment

In [21]:
network = "custom"  # set this to one of mainnet, yeouido, euljiro, pagoda, or custom
contracts_file_custom = 'custom_contracts_20201125111607.pkl'
#contracts_20201109120425

connections = {
"mainnet": {"iconservice": "https://ctz.solidwallet.io",       "nid": 1},
"yeouido": {"iconservice": "https://bicon.net.solidwallet.io", "nid": 3},
"euljiro": {"iconservice": "https://test-ctz.solidwallet.io",  "nid": 2},
"pagoda":  {"iconservice": "https://zicon.net.solidwallet.io", "nid":80},
"custom":  {"iconservice": "http://18.237.205.52:9000/",       "nid": 3},
"local":   {"iconservice": "http://localhost:9000/",           "nid": 3}
}

env = connections[network]

In [22]:
class AutoVivification(dict):
    """Implementation of perl's autovivification feature."""
    def __getitem__(self, item):
        try:
            return dict.__getitem__(self, item)
        except KeyError:
            value = self[item] = type(self)()
            return value

In [23]:
from iconsdk.exception import JSONRPCException
from iconsdk.libs.in_memory_zip import gen_deploy_data_content
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
from datetime import datetime
from repeater import retry
from shutil import make_archive
import requests
import json
import pickle as pkl
import csv
import os
from pprint import pprint
from getpass import getpass


ICX = 1000000000000000000 # 10**18
GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"


with open(contracts_file_custom, 'rb') as f:
    contracts = pkl.load(f)

@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result




### Required service

In [24]:
icon_service = IconService(HTTPProvider(env["iconservice"], 3))
NID = env["nid"]

In [25]:
pprint(contracts)

{'addressProvider': {'SCORE': 'cx2a4267624dfeb9a79b931424fd8ac7b81c5cf805',
                     'zip': '/home/elite/Desktop/OMM '
                            'OG/openmoneymarket-mono/score/addressProvider'},
 'feeProvider': {'SCORE': 'cx14ed4464849d2b34a43a54e4ae8bbdcf64bae671',
                 'zip': '/home/elite/Desktop/OMM '
                        'OG/openmoneymarket-mono/score/feeProvider'},
 'lendingPool': {'SCORE': 'cx5dbae1f5dc20f2d0d16034a07869b6f0bedd14f6',
                 'zip': '/home/elite/Desktop/OMM '
                        'OG/openmoneymarket-mono/score/lendingPool'},
 'lendingPoolCore': {'SCORE': 'cxaa345a5dbea93b27754e44024d5ea14ccbf09c62',
                     'zip': '/home/elite/Desktop/OMM '
                            'OG/openmoneymarket-mono/score/lendingPoolCore'},
 'lendingPoolDataProvider': {'SCORE': 'cxb5083e61b035b31b0bffa7350ff276349069eb60',
                             'zip': '/home/elite/Desktop/OMM '
                                    'OG/openmoney

## Run this to configure contracts zip to your local file

In [69]:
directory=os.getcwd()
print('directory',directory)
newContracts = []
for contract in contracts:
    newContracts.append(contract)
for i in range(len(contracts)):
    name = newContracts[i]
    contracts[name]['zip'] = directory + "/" + name
pprint(contracts)

directory /home/elite/Desktop/OMM OG/openmoneymarket-mono/score
{'addressProvider': {'SCORE': 'cx1e6df31409cf3f68fe4028848dca876314a5fbfc',
                     'zip': '/home/elite/Desktop/OMM '
                            'OG/openmoneymarket-mono/score/addressProvider'},
 'feeProvider': {'SCORE': 'cxe408b46a010ff5f82e9e45610a0aadb4ff2b073c',
                 'zip': '/home/elite/Desktop/OMM '
                        'OG/openmoneymarket-mono/score/feeProvider'},
 'lendingPool': {'SCORE': 'cxbfd0ecb9892ca2e1ae3d0494d48f6618620b7e48',
                 'zip': '/home/elite/Desktop/OMM '
                        'OG/openmoneymarket-mono/score/lendingPool'},
 'lendingPoolCore': {'SCORE': 'cx3daa5c2517fdd44bdf37904e5e537eefafb4d69d',
                     'zip': '/home/elite/Desktop/OMM '
                            'OG/openmoneymarket-mono/score/lendingPoolCore'},
 'lendingPoolDataProvider': {'SCORE': 'cx726e8bacc442256099dda50438c11d004215afc5',
                             'zip': '/home/elite

# Importing wallet

### Importing from private key

In [26]:

private="a691ef7d5601f9b5be4f9b9d80215159ea6ff0b88003e3d34e078d40e778b39a"
deployer_wallet = KeyWallet.load(bytes.fromhex(private))
deployer_wallet.get_address()


'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce'

In [7]:
icon_service.get_balance("hx91bf040426f226b3bfcd2f0b5967bbb0320525ce")

78702736970265600000000000

### Importing wallet from keystore file

In [8]:
# Set to the address of the keystore file and give the password when prompted
password = getpass()
deployer_wallet = KeyWallet.load("./keystores/keystore_test1.json",password)
deployer_wallet.get_address()
icon_service.get_balance(deployer_wallet.get_address())

708876575999000000000000000

### Creating new wallet

In [ ]:
deployer_wallet = KeyWallet.create()
print(deployer_wallet.get_address())
caller_wallet = KeyWallet.create()
print(caller_wallet.get_address())

In [94]:
# Add the contracts. Copy the output from this cell the the one above to save the contract addresses.
#Run this when you need to zip the contract codes (before deploying them)
contracts=AutoVivification()

deploy = ['addressProvider', 'feeProvider', 'lendingPool', 'lendingPoolCore', 'lendingPoolDataProvider','oToken','priceOracle','sample_token', 'sicx']
# for directory in {"score"}:
directory=os.getcwd()
with os.scandir(directory) as it:
    for file in it:
        archive_name = directory + "/" + file.name
        if file.is_dir() and file.name in deploy:
            # make_archive(archive_name, "zip", directory, file.name)
            contracts[file.name]['zip'] = archive_name 
contracts['oICX']['zip'] = directory + "/oToken"

contracts

{'oToken': {'zip': '/home/elite/Desktop/OMM OG/openmoneymarket-mono/score/oToken'},
 'sicx': {'zip': '/home/elite/Desktop/OMM OG/openmoneymarket-mono/score/sicx'},
 'addressProvider': {'zip': '/home/elite/Desktop/OMM OG/openmoneymarket-mono/score/addressProvider'},
 'lendingPoolDataProvider': {'zip': '/home/elite/Desktop/OMM OG/openmoneymarket-mono/score/lendingPoolDataProvider'},
 'priceOracle': {'zip': '/home/elite/Desktop/OMM OG/openmoneymarket-mono/score/priceOracle'},
 'sample_token': {'zip': '/home/elite/Desktop/OMM OG/openmoneymarket-mono/score/sample_token'},
 'lendingPoolCore': {'zip': '/home/elite/Desktop/OMM OG/openmoneymarket-mono/score/lendingPoolCore'},
 'lendingPool': {'zip': '/home/elite/Desktop/OMM OG/openmoneymarket-mono/score/lendingPool'},
 'feeProvider': {'zip': '/home/elite/Desktop/OMM OG/openmoneymarket-mono/score/feeProvider'},
 'oICX': {'zip': '/home/elite/Desktop/OMM OG/openmoneymarket-mono/score/oToken'}}

# Deploying all contracts


In [95]:
#this cell needs to run only when you need to deploy new set of contracts
for score, loc in contracts.items():
    params = {}
    print(score,loc)
    if score == "sample_token":
        params = {'_initialSupply':500000000, '_decimals': 18}
    elif score == "sicx":
        params = {'_initialSupply':500000000, '_decimals': 18}
    elif score == "oICX":
        params = {"_name":"BridgeICXInterestToken","_symbol":"oICX"}
    elif score == "oToken":
        params = {"_name":"BridgeUSDInterestToken","_symbol":"oUSDb"}
    deploy_transaction = DeployTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(GOVERNANCE_ADDRESS)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content(loc["zip"]))\
        .params(params)\
        .build()

    step_limit = icon_service.estimate_step(deploy_transaction) + 100000
    
    signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
    tx_hash = icon_service.send_transaction(signed_transaction)

    res = get_tx_result(tx_hash)
    contracts[score]["SCORE"] = res.get('scoreAddress', '')
now = datetime.utcnow()
filename = f'contracts_{now.year}{str(now.month).zfill(2)}{str(now.day).zfill(2)}{str(now.hour).zfill(2)}{str(now.minute).zfill(2)}{str(now.second).zfill(2)}.pkl'
filename=network+'_'+filename
print(f'saving contracts list as {filename}')
with open(filename, 'wb') as f:
    pkl.dump(contracts, f)
contracts

oToken {'zip': '/home/elite/Desktop/OMM OG/openmoneymarket-mono/score/oToken'}
sicx {'zip': '/home/elite/Desktop/OMM OG/openmoneymarket-mono/score/sicx'}
addressProvider {'zip': '/home/elite/Desktop/OMM OG/openmoneymarket-mono/score/addressProvider'}
lendingPoolDataProvider {'zip': '/home/elite/Desktop/OMM OG/openmoneymarket-mono/score/lendingPoolDataProvider'}
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
priceOracle {'zip': '/home/elite/Desktop/OMM OG/openmoneymarket-mono/score/priceOracle'}
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
sample_token {'zip': '/home/elite/Desktop/OMM OG/openmoneymarket-mono/score/sample_token'}
Raised JSONRPC

{'oToken': {'zip': '/home/elite/Desktop/OMM OG/openmoneymarket-mono/score/oToken',
  'SCORE': 'cxc3011b3da8a3df88e9411efefe1207c6d3471e65'},
 'sicx': {'zip': '/home/elite/Desktop/OMM OG/openmoneymarket-mono/score/sicx',
  'SCORE': 'cx2d53c5e8347915733500bda8bdf2aabefdd833d9'},
 'addressProvider': {'zip': '/home/elite/Desktop/OMM OG/openmoneymarket-mono/score/addressProvider',
  'SCORE': 'cx5f2a0a21f217d032e81b70990de4b10fbba7ca99'},
 'lendingPoolDataProvider': {'zip': '/home/elite/Desktop/OMM OG/openmoneymarket-mono/score/lendingPoolDataProvider',
  'SCORE': 'cx32586b06dc00d24e1001e0b689353184d61fb3ad'},
 'priceOracle': {'zip': '/home/elite/Desktop/OMM OG/openmoneymarket-mono/score/priceOracle',
  'SCORE': 'cx03d534d6fbd8a43ca8d7385c16758072e342bcd4'},
 'sample_token': {'zip': '/home/elite/Desktop/OMM OG/openmoneymarket-mono/score/sample_token',
  'SCORE': 'cxf27f51fe6a092437fd54c5f74983391515dfec2d'},
 'lendingPoolCore': {'zip': '/home/elite/Desktop/OMM OG/openmoneymarket-mono/score/l

## Save updated contract

In [46]:
#use this to update a pre existing pkl file
filename = 'updated20201125_' + contracts_file_custom
print(f'saving contracts list as {filename}')
with open(filename, 'wb') as f:
    pkl.dump(contracts, f)
contracts

saving contracts list as updated20201125_custom_contracts_20201122161857.pkl


{'lendingPoolDataProvider': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/lendingPoolDataProvider',
  'SCORE': 'cx726e8bacc442256099dda50438c11d004215afc5'},
 'lendingPool': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/lendingPool',
  'SCORE': 'cxbfd0ecb9892ca2e1ae3d0494d48f6618620b7e48'},
 'feeProvider': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/feeProvider',
  'SCORE': 'cxe408b46a010ff5f82e9e45610a0aadb4ff2b073c'},
 'lendingPoolCore': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/lendingPoolCore',
  'SCORE': 'cx3daa5c2517fdd44bdf37904e5e537eefafb4d69d'},
 'oToken': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/oToken',
  'SCORE': 'cx2f4c43ec2c69321573e93549d89462de4a25fdbe'},
 'priceOracle': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/priceOracle',
  'SCORE': 'cx6703fdc7e0e65c5195d6dbfb59a838f76a9c19b1'},
 'sample_token': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/sample_token'

In [18]:
wallet_address='hx49c8a4c31e84b9abe7f8c8395eb7133b2151aeae'
balance=icon_service.get_balance(wallet_address)
print(balance/10**18)

1000000.0


# ICX transfer

In [17]:
transaction = TransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to("hx49c8a4c31e84b9abe7f8c8395eb7133b2151aeae")\
    .nid(3) \
    .nonce(100) \
    .value(1000000*10**18)\
    .build()

estimate_step = icon_service.estimate_step(transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(transaction, deployer_wallet,step_limit)


# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

{'txHash': '0x3c546494f559d4ea9d34534f13387c74aa80658c67ac2630ce1adaf514e1bc86',
 'blockHeight': 1425621,
 'blockHash': '0xc6a2b22340f3c2756027f1926f731d1eaa78ed6c65653e92e85f58e7ee2d2b66',
 'txIndex': 1,
 'to': 'hx49c8a4c31e84b9abe7f8c8395eb7133b2151aeae',
 'stepUsed': 100000,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 100000,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

In [ ]:
params = {"_owner":"hx91bf040426f226b3bfcd2f0b5967bbb0320525ce"}
transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['sample_token']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(3)\
    .nonce(100)\
    .method("balanceOf")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

In [20]:
params = {"_owner":"hx91bf040426f226b3bfcd2f0b5967bbb0320525ce"}
transaction = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['sample_token']['SCORE'])\
    .method("balanceOf")\
    .params(params)\
    .build()
response=icon_service.call(transaction)
print(response)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32002, 'message': 'SCORE not found: cx19be1be7b4e7750863edf966faac3f42dec21025'}


JSONRPCException: {'code': -32002, 'message': 'SCORE not found: cx19be1be7b4e7750863edf966faac3f42dec21025'} (Json rpc error)

# Deploying otoken and sicx for icx

In [31]:
params = {"_name":"ICXinterestToken","_symbol":"oICX"}
deploy_transaction = DeployTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(GOVERNANCE_ADDRESS)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content('oToken'))\
        .params(params)\
        .build()

step_limit = icon_service.estimate_step(deploy_transaction) + 100000

signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(res)
contracts['oICX']["SCORE"] = res.get('scoreAddress', '')
contracts['oICX']["zip"] = '/home/shubham/Work/ICON/openmoneymarket-mono/score/oToken'

params = {"_name":"ICXinterestToken","_symbol":"oICX"}
deploy_transaction = DeployTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(GOVERNANCE_ADDRESS)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content('oToken'))\
        .params(params)\
        .build()

step_limit = icon_service.estimate_step(deploy_transaction) + 100000

signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(res)
contracts['sicx']["SCORE"] = res.get('scoreAddress', '')
contracts['sicx']["zip"] = '/home/shubham/Work/ICON/openmoneymarket-mono/score/sicx'

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
{'txHash': '0xb1ae2334359e4fb41741ac4bbf66cc9166ee4daeafac8fb2885d800fa62c554c', 'blockHeight': 1128147, 'blockHash': '0xc836630e1d11925b0b4286f38e469d779301c15aa98994c5d795070aa1955a44', 'txIndex': 1, 'to': 'cx0000000000000000000000000000000000000000', 'scoreAddress': 'cx6dbb85df52d7b12d2c00d673d89c5bf21ee29be2', 'stepUsed': 1144284440, 'stepPrice': 10000000000, 'cumulativeStepUsed': 1144284440, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

In [77]:
pprint(contracts)
print(contracts['lendingPool']['zip'])

{'addressProvider': {'SCORE': 'cx1e6df31409cf3f68fe4028848dca876314a5fbfc',
                     'zip': '/home/elite/Desktop/OMM '
                            'OG/openmoneymarket-mono/score/addressProvider'},
 'feeProvider': {'SCORE': 'cxe408b46a010ff5f82e9e45610a0aadb4ff2b073c',
                 'zip': '/home/elite/Desktop/OMM '
                        'OG/openmoneymarket-mono/score/feeProvider'},
 'lendingPool': {'SCORE': 'cxbfd0ecb9892ca2e1ae3d0494d48f6618620b7e48',
                 'zip': '/home/elite/Desktop/OMM '
                        'OG/openmoneymarket-mono/score/lendingPool'},
 'lendingPoolCore': {'SCORE': 'cx3daa5c2517fdd44bdf37904e5e537eefafb4d69d',
                     'zip': '/home/elite/Desktop/OMM '
                            'OG/openmoneymarket-mono/score/lendingPoolCore'},
 'lendingPoolDataProvider': {'SCORE': 'cx726e8bacc442256099dda50438c11d004215afc5',
                             'zip': '/home/elite/Desktop/OMM '
                                    'OG/openmoney

# Updating a contract

In [28]:
contract = 'lendingPoolDataProvider'
update = contracts[contract]['SCORE']
deploy_transaction = DeployTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(update)\
    .nid(NID)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content('lendingPoolDataProvider'))\
    .build()

step_limit = icon_service.estimate_step(deploy_transaction) + 100000

signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0x9a68899a28320df72b364b52060cb6f7d234fe7b771c9e4553c94287eaf1da8d',
 'blockHeight': 1433914,
 'blockHash': '0xc78d496f8de8aad0e2c9e5595bebd58bef8f823a448f7aff8b3ec69ef1cd2de7',
 'txIndex': 1,
 'to': 'cxb5083e61b035b31b0bffa7350ff276349069eb60',
 'scoreAddress': 'cxb5083e61b035b31b0bffa7350ff276349069eb60',
 'stepUsed': 1705749600,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 1705749600,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

# Configuring SCOREs's parameters

In [101]:
settings = [{'contract': 'lendingPool', 'method': 'setLendingPoolCoreAddress', 'params':{'_address': contracts['lendingPoolCore']['SCORE']}},
            {'contract': 'lendingPool', 'method': 'setUSDbAddress', 'params':{'_address': contracts['sample_token']['SCORE']}},
            {'contract': 'lendingPool', 'method': 'setDataProvider', 'params':{'_address': contracts['lendingPoolDataProvider']['SCORE']}},
            {'contract': 'lendingPool', 'method': 'setFeeProvider', 'params':{'_address': contracts['feeProvider']['SCORE']}},
            {'contract': 'lendingPool', 'method': 'setSicxAddress', 'params':{'_address': contracts['sicx']['SCORE']}},
            {'contract': 'feeProvider', 'method': 'setLoanOriginationFeePercentage', 'params':{'_percentage': 1*10**16}},
            {'contract': 'lendingPoolDataProvider', 'method': 'setSymbol', 'params':{'_reserveAddress': contracts['sample_token']['SCORE'],'_sym':"USDb"}},
            {'contract': 'lendingPoolDataProvider', 'method': 'setLendingPoolCoreAddress', 'params':{'_address': contracts['lendingPoolCore']['SCORE']}},
            {'contract': 'lendingPoolDataProvider', 'method': 'setOracleAddress', 'params':{'_address': contracts['priceOracle']['SCORE']}},
            {'contract': 'oToken', 'method': 'setCoreAddress', 'params':{'_address':contracts['lendingPoolCore']['SCORE']}},
            {'contract': 'oToken', 'method': 'setReserveAddress', 'params':{'_address':contracts['sample_token']['SCORE']}},
            {'contract': 'oToken', 'method': 'setDataProviderAddress', 'params':{'_address':contracts['lendingPoolDataProvider']['SCORE']}},
            {'contract': 'oToken', 'method': 'setLendingPoolAddress', 'params':{'_address':contracts['lendingPool']['SCORE']}},
            {'contract': 'priceOracle', 'method': 'set_reference_data', 'params':{'_base':'USDb','_quote':'USD','_rate':1*10**18}},
            {'contract': 'addressProvider', 'method': 'setLendingPool', 'params':{'_address':contracts['lendingPool']['SCORE']}},
            {'contract': 'addressProvider', 'method': 'setLendingPoolDataProvider', 'params':{'_address':contracts['lendingPoolDataProvider']['SCORE']}},
            {'contract': 'addressProvider', 'method': 'setUSDb', 'params':{'_address':contracts['sample_token']['SCORE']}},
            {'contract': 'addressProvider', 'method': 'setoUSDb', 'params':{'_address':contracts['oToken']['SCORE']}},
            {'contract': 'lendingPoolCore', 'method': 'setReserveConstants', 'params' :{"_constants": [{"reserve":contracts['sample_token']['SCORE'],"optimalUtilizationRate":f"6{'0'*17}","baseBorrowRate":f"1{'0'*16}","slopeRate1":f"4{'0'*16}","slopeRate2":f"5{'0'*17}"} ]}}]

In [29]:
settings=[{'contract': 'feeProvider', 'method': 'setLoanOriginationFeePercentage', 'params':{'_percentage': 1*10**16}}]

In [30]:
for sett in settings:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']]['SCORE'])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash))



Calling setLoanOriginationFeePercentage, with parameters {'_percentage': 10000000000000000} on the feeProvider contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
{'txHash': '0xe29f161f36639afbf6919b8b755543b93fe5dd41a40e43457578fa253f211679', 'blockHeight': 1434658, 'blockHash': '0x1ecb66691745086eed6f7a73f39d689edfc3ceb431678ad9ad2bffccece89bfa', 'txIndex': 1, 'to': 'cx14ed4464849d2b34a43a54e4ae8bbdcf64bae671', 'stepUsed': 143160, 'stepPrice': 10000000000, 'cumulativeStepUsed': 143160, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

## Additional setup for icx reserve

In [103]:
icx_additional=[{'contract': 'lendingPool', 'method': 'setSICXAddress', 'params':{'_address': contracts['sicx']['SCORE']}},
                {'contract': 'priceOracle', 'method': 'set_reference_data', 'params':{'_base':'sICX','_quote':'USD','_rate':5*10**17}},
                {'contract': 'lendingPoolDataProvider', 'method': 'setSymbol', 'params':{'_reserveAddress': contracts['sicx']['SCORE'],'_sym':"sICX"}},
                {'contract': 'addressProvider', 'method': 'setsICX', 'params':{'_address':contracts['sicx']['SCORE']}},
                {'contract': 'addressProvider', 'method': 'setoICX', 'params':{'_address':contracts['oICX']['SCORE']}},
                {'contract': 'lendingPoolCore', 'method': 'setReserveConstants', 'params' :{"_constants": [{"reserve":contracts['sicx']['SCORE'],"optimalUtilizationRate":f"6{'0'*17}","baseBorrowRate":f"0{'0'*17}","slopeRate1":f"7{'0'*16}","slopeRate2":f"3{'0'*18}"} ]}},
                 
                {'contract': 'oICX', 'method': 'setCoreAddress', 'params':{'_address':contracts['lendingPoolCore']['SCORE']}},
                {'contract': 'oICX', 'method': 'setReserveAddress', 'params':{'_address':contracts['sicx']['SCORE']}},
                {'contract': 'oICX', 'method': 'setDataProviderAddress', 'params':{'_address':contracts['lendingPoolDataProvider']['SCORE']}},
                {'contract': 'oICX', 'method': 'setLendingPoolAddress', 'params':{'_address':contracts['lendingPool']['SCORE']}},


]

In [104]:
for sett in icx_additional:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']]['SCORE'])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash))



Calling setSICXAddress, with parameters {'_address': 'cx2d53c5e8347915733500bda8bdf2aabefdd833d9'} on the lendingPool contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
{'txHash': '0x78b6e454b09ebbad6e9062b75038c26613a8e69e83b6dcc37f64ed746823c140', 'blockHeight': 1353620, 'blockHash': '0x8c0d1851924354c9764b4e9a60fa893f9a0967506db9fdf172e49beeb3db5279', 'txIndex': 1, 'to': 'cx43184421f5f5743ae5435f59f12720e5b6c9a2c1', 'stepUsed': 150520, 'stepPrice': 10000000000, 'cumulativeStepUsed': 150520, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

## Adding USDb reserve in LendingPoolCore



In [105]:
params ={"_reserve": {"reserveAddress":contracts['sample_token']['SCORE'],"oTokenAddress":contracts['oToken']['SCORE'],"totalBorrows":"0","lastUpdateTimestamp": "0","liquidityRate":"0","borrowRate":"0","liquidityCumulativeIndex":f"1{'0'*18}","borrowCumulativeIndex":f"1{'0'*18}","baseLTVasCollateral":"600000000000000000","liquidationThreshold":"650000000000000000","liquidationBonus":"10","decimals":"18","borrowingEnabled": "1","usageAsCollateralEnabled":"1","isFreezed":"0","isActive":"1"} }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolCore']['SCORE']) \
    .nid(NID) \
    .nonce(100) \
    .method("addReserveData")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
# print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0x9091b4ea7ca9a8c53f3d7ec49ce4c2e3a6727aba95f629745531df2841022f04',
 'blockHeight': 1353642,
 'blockHash': '0x59db7e5cd9dd019ba0a2447f39986fd3b4bb581d8fbf9dd23c98a31f7b0ea270',
 'txIndex': 1,
 'to': 'cx6146eab7f96fa77400d4c9e9884f0ddc65e01855',
 'stepUsed': 271600,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 271600,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

## Adding ICX reserve

In [106]:
params ={"_reserve": {"reserveAddress":contracts['sicx']['SCORE'],"oTokenAddress":contracts['oICX']['SCORE'],"totalBorrows":"0","lastUpdateTimestamp": "0","liquidityRate":"0","borrowRate":"0","liquidityCumulativeIndex":f"1{'0'*18}","borrowCumulativeIndex":f"1{'0'*18}","baseLTVasCollateral":"330000000000000000","liquidationThreshold":"650000000000000000","liquidationBonus":"10","decimals":"18","borrowingEnabled": "1","usageAsCollateralEnabled":"1","isFreezed":"0","isActive":"1"} }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolCore']['SCORE']) \
    .nid(NID) \
    .nonce(100) \
    .method("addReserveData")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
# print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0x0ca7769d0458bd1b553b83a5772cd7bd923c3c249942eafceec863e5258c4a7f',
 'blockHeight': 1353645,
 'blockHash': '0x3d724103e80b38cfcc833b5d54706fd59b86429f51f2d42d082f43fc5989f284',
 'txIndex': 1,
 'to': 'cx6146eab7f96fa77400d4c9e9884f0ddc65e01855',
 'stepUsed': 271360,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 271360,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

# Transfering USDb to a wallet

In [107]:
params = {"_to": "hx91bf040426f226b3bfcd2f0b5967bbb0320525ce", "_value": 1000000000000000000000,}
transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['sample_token']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(3)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0x6ef1cc84e351239d9b54023facd39dbce650bd65260c11ed80daa06f577f7287',
 'blockHeight': 1353648,
 'blockHash': '0xb76dfd02caa3182e3db55c8415698635426d6365e213a40a35073a0d67d02b3b',
 'txIndex': 1,
 'to': 'cxf27f51fe6a092437fd54c5f74983391515dfec2d',
 'stepUsed': 158920,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 158920,
 'eventLogs': [{'scoreAddress': 'cxf27f51fe6a092437fd54c5f74983391515dfec2d',
   'indexed': ['Transfer(Address,Address,int,bytes)',
    'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce',
    'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce',
    '0x3635c9adc5dea00000'],
   'data': ['0x4e6f6e65']}],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x10\x00\x00\x00 \x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x80\x00\x00\x00\x00\x00\x00\x00\x80\x00\x00\x80\x00\x08\x00\x00\x00\x00\x00 \x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

# Depositing USDb from a wallet

In [109]:
data = "{\"method\": \"deposit\", \"params\": {\"amount\": 1000000000000000000000}}".encode("utf-8")
params = {"_to": contracts['lendingPool']['SCORE'], "_value": 1000000000000000000000, "_data": data}
transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['sample_token']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(3)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

{'txHash': '0xd61dd328e5dfa251632609893233dd383a15c76e680cf0b5f4d8751867e6340a',
 'blockHeight': 1353677,
 'blockHash': '0xa0b52c7c33c17a11f4d114baab6c9a18c87e500fd8e375cdaa00395c2b8aba9c',
 'txIndex': 1,
 'to': 'cxf27f51fe6a092437fd54c5f74983391515dfec2d',
 'stepUsed': 751720,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 751720,
 'eventLogs': [{'scoreAddress': 'cx6146eab7f96fa77400d4c9e9884f0ddc65e01855',
   'indexed': ['PrintData(str,int,int,int)',
    'params check core line 408',
    '0x3635c9adc5dea00000',
    '0x0'],
   'data': ['0x0']},
  {'scoreAddress': 'cx6146eab7f96fa77400d4c9e9884f0ddc65e01855',
   'indexed': ['PrintData(str,int,int,int)',
    'rate value core line 412',
    '0x0',
    '0x2386f26fc10000'],
   'data': ['0x0']},
  {'scoreAddress': 'cx6146eab7f96fa77400d4c9e9884f0ddc65e01855',
   'indexed': ['ReserveUpdated(Address,int,int,int,int)',
    'cxf27f51fe6a092437fd54c5f74983391515dfec2d',
    '0x0',
    '0x2386f26fc10000'],
   'data': ['0xde0b6b3a7640000', '0xd

# Borrowing USDb

In [110]:
params ={"_reserve": contracts['sample_token']['SCORE'], "_amount":10*10**18 }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPool']['SCORE']) \
    .nid(NID) \
    .nonce(100) \
    .method("borrow")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

{'version': '0x3', 'from': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce', 'to': 'cx43184421f5f5743ae5435f59f12720e5b6c9a2c1', 'stepLimit': '0x146dc4', 'timestamp': '0x5b553bdb121c6', 'nid': '0x3', 'nonce': '0x64', 'dataType': 'call', 'data': {'method': 'borrow', 'params': {'_reserve': 'cxf27f51fe6a092437fd54c5f74983391515dfec2d', '_amount': '0x8ac7230489e80000'}}, 'signature': 'yxSWWut7ryf4/6NtDopmBxaHiCw+r5dhQzzezt7x14YKG0fSVISEjPYE6Timllqvv4lfWcJ4RD+A7/ww7J3/5gE='}
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0xd14ec29c4c2ecb85012d82d6882410d9399c4a41da0a560a472262781a009594',
 'blockHeight': 1353685,
 'blockHash': '0x0ccccb53393581722268081256f49cd12598f808c55a12c9189e73283682de1b',
 'txIndex': 1,
 'to': 'cx43184421f5f5743ae5435f59f12720e5b6c9a2c1',
 'stepUsed': 1238820,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 1238820,
 'eventLogs': [{'scoreAddress': 'cx43184421f5f5743ae5435f59f12720e5b6c9a2c1',
   'indexed': ['PrintData(str,int,int,int)',
    'available liquidity at pool line 245',
    '0x3635c9adc5dea00000',
    '0x0'],
   'data': ['0x0']},
  {'scoreAddress': 'cx43184421f5f5743ae5435f59f12720e5b6c9a2c1',
   'indexed': ['PrintData(str,int,int,int)',
    'data at pool line 256',
    '0x3635c9adc5dea00000',
    '0x0'],
   'data': ['0x0']},
  {'scoreAddress': 'cx43184421f5f5743ae5435f59f12720e5b6c9a2c1',
   'indexed': ['PrintData(str,int,int,int)',
    'amout of collateral needed USD pool line 268',
    '0xe74be5079082aaab',
    '0x0'],
   'data': ['0x0']},
  {'scoreAdd

# Reedeming/Withdrawing USDb

In [111]:
params ={"_amount": 10*10**18 }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['oToken']['SCORE']) \
    .nid(NID) \
    .nonce(100) \
    .method("redeem")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

{'version': '0x3', 'from': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce', 'to': 'cxc3011b3da8a3df88e9411efefe1207c6d3471e65', 'stepLimit': '0x143ebc', 'timestamp': '0x5b553bf01f180', 'nid': '0x3', 'nonce': '0x64', 'dataType': 'call', 'data': {'method': 'redeem', 'params': {'_amount': '0x8ac7230489e80000'}}, 'signature': 'A6J8pKu1O6z4b6+yRLpfqwyJ4duW7yxDUyS/z0tw5TRaYflRZ4z98yS881ACeFNCL+V/TjvTpJGOWm35iNIKoQE='}
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0x5b0cc66a6ff064bf2ecc8d9d50b0512570308396c11a958925a403a1ec357a5f',
 'blockHeight': 1353696,
 'blockHash': '0x2a41466d89cd9b869a0500d849a6422bf8e41065b272106100973ce157886430',
 'txIndex': 1,
 'to': 'cxc3011b3da8a3df88e9411efefe1207c6d3471e65',
 'stepUsed': 1226780,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 1226780,
 'eventLogs': [{'scoreAddress': 'cxc3011b3da8a3df88e9411efefe1207c6d3471e65',
   'indexed': ['Mint(Address,int)',
    'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce'],
   'data': ['0xf97c99928']},
  {'scoreAddress': 'cxc3011b3da8a3df88e9411efefe1207c6d3471e65',
   'indexed': ['Burn(Address,int)',
    'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce'],
   'data': ['0x8ac7230489e80000']},
  {'scoreAddress': 'cx6146eab7f96fa77400d4c9e9884f0ddc65e01855',
   'indexed': ['PrintData(str,int,int,int)',
    'params check core line 408',
    '0x0',
    '0x8ac7230489e80000'],
   'data': ['0x0']},
  {'scoreAddress': 'cx6146eab7f96fa77400d4c9e9884f0ddc65e01855',
   'indexed

In [112]:
params ={"_reserveAddress": contracts['sample_token']['SCORE'],"_baseLTVasCollateral" :33*10**16}
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolCore']['SCORE']) \
    .nid(NID) \
    .nonce(100) \
    .method("updateBaseLTVasCollateral")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

{'version': '0x3', 'from': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce', 'to': 'cx6146eab7f96fa77400d4c9e9884f0ddc65e01855', 'stepLimit': '0x3ea80', 'timestamp': '0x5b553c06ebdc6', 'nid': '0x3', 'nonce': '0x64', 'dataType': 'call', 'data': {'method': 'updateBaseLTVasCollateral', 'params': {'_reserveAddress': 'cxf27f51fe6a092437fd54c5f74983391515dfec2d', '_baseLTVasCollateral': '0x494654067e10000'}}, 'signature': '79de8WIdF8kiL8fPiEdEOhCZE8E3ccr1MEVFQLQv1G8+mWWe7G22aAzjG93wIm6O9OG3xu1QOnEF0vcSfNApbAA='}
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0x9b656a472068de6e8f2b40f3003b4cfcae8fb206557ecef3a00da84c67d816c7',
 'blockHeight': 1353708,
 'blockHash': '0xd794c262f864c4f681a19ed983b8b36cb5f855cdaf0244ecc6ab7b1cace2fcef',
 'txIndex': 1,
 'to': 'cx6146eab7f96fa77400d4c9e9884f0ddc65e01855',
 'stepUsed': 156640,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 156640,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

# Repaying USDb

In [113]:
data = "{\"method\": \"repay\", \"params\": {\"amount\": 10000000000000000000}}".encode("utf-8")
params = {"_to": contracts['lendingPool']['SCORE'], "_value": 10000000000000000000, "_data": data}
transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['sample_token']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(3)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0x277fae145ad2b5aba3b4658c75f9c795b9cd26d2a014e803f440349f706b0ed5',
 'blockHeight': 1353714,
 'blockHash': '0x807be102e3e53321d0192a4fea9efc26acedd859277c2fcfe785651ebea33f5b',
 'txIndex': 1,
 'to': 'cxf27f51fe6a092437fd54c5f74983391515dfec2d',
 'stepUsed': 929860,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 929860,
 'eventLogs': [{'scoreAddress': 'cx6146eab7f96fa77400d4c9e9884f0ddc65e01855',
   'indexed': ['PrintData(str,int,int,int)',
    'params check core line 408',
    '0x8a6e51a672858000',
    '0x0'],
   'data': ['0x0']},
  {'scoreAddress': 'cx6146eab7f96fa77400d4c9e9884f0ddc65e01855',
   'indexed': ['PrintData(str,int,int,int)',
    'rate value core line 412',
    '0x34ece5a8f5',
    '0x23887a690fba23'],
   'data': ['0x0']},
  {'scoreAddress': 'cx6146eab7f96fa77400d4c9e9884f0ddc65e01855',
   'indexed': ['ReserveUpdated(Address,int,int,int,int)',
    'cxf27f51fe6a092437fd54c5f74983391515dfec2d',
    '0x34ece5a8f5',
    '0x23887a690fba23'],
   'data': ['0xde0b6b

In [23]:
print(icon_service.get_balance("hx91bf040426f226b3bfcd2f0b5967bbb0320525ce"))

9450000385630198414715


# Depositing ICX from Wallet

In [114]:
params = {"_amount": 500 * 10 ** 18}
call_transaction = CallTransactionBuilder() \
    .from_(deployer_wallet.get_address()) \
    .to(contracts['lendingPool']['SCORE']) \
    .nid(3) \
    .step_limit(1000000000) \
    .nonce(100) \
    .value(500 * 10 ** 18) \
    .method("deposit") \
    .params(params) \
    .build() 

signed_transaction = SignedTransaction(call_transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0x5142a4c6b2ed37986f2e06689ec53d7e884a8cd7ccdced40f64e4f482261d568',
 'blockHeight': 1353718,
 'blockHash': '0x1ce2ce8efae957061913479ec271b5a504a5049d594020d57a18704130fe3de3',
 'txIndex': 1,
 'to': 'cx43184421f5f5743ae5435f59f12720e5b6c9a2c1',
 'stepUsed': 571080,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 571080,
 'eventLogs': [{'scoreAddress': 'cx43184421f5f5743ae5435f59f12720e5b6c9a2c1',
   'indexed': ['ICXTransfer(Address,Address,int)',
    'cx43184421f5f5743ae5435f59f12720e5b6c9a2c1',
    'cx2d53c5e8347915733500bda8bdf2aabefdd833d9',
    '0x1b1ae4d6e2ef500000'],
   'data': []},
  {'scoreAddress': 'cx2d53c5e8347915733500bda8bdf2aabefdd833d9',
   'indexed': ['Mint(Address,int)',
    'cx6146eab7f96fa77400d4c9e9884f0ddc65e01855',
    '0x1b1ae4d6e2ef500000'],
   'data': []},
  {'scoreAddress': 'cx6146eab7f96fa77400d4c9e9884f0ddc65e01855',
   'indexed': ['PrintData(str,int,int,int)',
    'params check core line 408',
    '0x1b1ae4d6e2ef500000',
    '0x0'],
   'data'

# Withdrawing ICX 

In [115]:
params = {"_amount": 250 * 10 ** 18}
call_transaction = CallTransactionBuilder() \
    .from_(deployer_wallet.get_address())\
    .to(contracts['oICX']['SCORE'])\
    .nid(3) \
    .step_limit(10000000) \
    .nonce(100) \
    .method("redeem") \
    .params(params) \
    .build()
signed_transaction = SignedTransaction(call_transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)
        

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0x4e18d42ba4451db152afef7509f2ab5c4cbb0efb3e9123ef47930d2a2641721d',
 'blockHeight': 1353723,
 'blockHash': '0x7e9f49501066c8329b270e447d413c104bab52118103d07edbfff08f78f7e25a',
 'txIndex': 1,
 'to': 'cx91a8f04a2a5adcf110168e68479e34a3b33424c9',
 'stepUsed': 1396820,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 1396820,
 'eventLogs': [{'scoreAddress': 'cx91a8f04a2a5adcf110168e68479e34a3b33424c9',
   'indexed': ['Mint(Address,int)',
    'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce'],
   'data': ['0x0']},
  {'scoreAddress': 'cx91a8f04a2a5adcf110168e68479e34a3b33424c9',
   'indexed': ['Burn(Address,int)',
    'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce'],
   'data': ['0xd8d726b7177a80000']},
  {'scoreAddress': 'cx6146eab7f96fa77400d4c9e9884f0ddc65e01855',
   'indexed': ['PrintData(str,int,int,int)',
    'params check core line 408',
    '0x0',
    '0xd8d726b7177a80000'],
   'data': ['0x0']},
  {'scoreAddress': 'cx6146eab7f96fa77400d4c9e9884f0ddc65e01855',
   'indexed': ['P

# Borrowing sicx

In [26]:
#borrow sicx
params ={"_reserve": contracts['sicx']['SCORE'], "_amount":100*10**18 }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPool']['SCORE']) \
    .nid(NID) \
    .nonce(100) \
    .method("borrow")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

{'version': '0x3', 'from': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce', 'to': 'cx35b12ca4e9c672518db3c06d92ed60d50e7d5554', 'stepLimit': '0x18a290', 'timestamp': '0x5b56a6cdd75f1', 'nid': '0x3', 'nonce': '0x64', 'dataType': 'call', 'data': {'method': 'borrow', 'params': {'_reserve': 'cx3e9d38175d112cd5cf05a90080cce779f202ba27', '_amount': '0x56bc75e2d63100000'}}, 'signature': 'FpOevHks4R+7T08A8dEF5XIXJgZljUnkjYvuFmke3ytj0fJBOrAu+BrUzZrMTFaKN2P38jH5WgrgHR7N0jnHywE='}
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Invalid params txHash'}
{'code': -32602, 'message': 'Invalid params txHash'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0x1f0f72eab1db758de1b644b7e1054de762de98699c7e2f5c4f840d01cc6fb92a',
 'blockHeight': 9908027,
 'blockHash': '0xd3b91a1a3a7cf36f636c3a0b57a6f6e7ac032add02a83002308a1415f7524cbd',
 'txIndex': 1,
 'to': 'cx35b12ca4e9c672518db3c06d92ed60d50e7d5554',
 'stepUsed': 1514480,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 1514480,
 'eventLogs': [{'scoreAddress': 'cx35b12ca4e9c672518db3c06d92ed60d50e7d5554',
   'indexed': ['PrintData(str,int,int,int)',
    'available liquidity at pool line 245',
    '0xba4414334ab230000',
    '0x0'],
   'data': ['0x0']},
  {'scoreAddress': 'cx35b12ca4e9c672518db3c06d92ed60d50e7d5554',
   'indexed': ['PrintData(str,int,int,int)',
    'data at pool line 256',
    '0x14c09b0f07ec6d8a819',
    '0x1174b6c563054415d'],
   'data': ['0x0']},
  {'scoreAddress': 'cx35b12ca4e9c672518db3c06d92ed60d50e7d5554',
   'indexed': ['PrintData(str,int,int,int)',
    'amout of collateral needed USD pool line 268',
    '0xb850adebe4f559f48',
    '0x0'],
   'data': ['0x0

# Repay sicx

In [29]:

data = "{\"method\": \"repay\", \"params\": {\"amount\": 100000000000000000000}}".encode("utf-8")
params = {"_to": contracts['lendingPool']['SCORE'], "_value": 100000000000000000000, "_data": data}
transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['sicx']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(3)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0x4b95d8683af8ba72d8ab4ba223c31b1c9c2f5e5b9ad262a4fa6f1a2ad45138f7',
 'blockHeight': 9908080,
 'blockHash': '0x51f885287ac717250a1775e9dc72600273acde6d78f3b4689fff9b3a40594a29',
 'txIndex': 1,
 'to': 'cx3e9d38175d112cd5cf05a90080cce779f202ba27',
 'stepUsed': 867000,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 867000,
 'eventLogs': [{'scoreAddress': 'cx327a386a9f8076045679fc132a8baa9e270ca43b',
   'indexed': ['PrintData(str,int,int,int)',
    'params check core line 408',
    '0x5684f308079370000',
    '0x0'],
   'data': ['0x0']},
  {'scoreAddress': 'cx327a386a9f8076045679fc132a8baa9e270ca43b',
   'indexed': ['PrintData(str,int,int,int)',
    'rate value core line 412',
    '0x9cdabb163294f',
    '0x433145f3bf1722'],
   'data': ['0x0']},
  {'scoreAddress': 'cx327a386a9f8076045679fc132a8baa9e270ca43b',
   'indexed': ['ReserveUpdated(Address,int,int,int,int)',
    'cx3e9d38175d112cd5cf05a90080cce779f202ba27',
    '0x9cdabb163294f',
    '0x433145f3bf1722'],
   'data': ['0

# Readonly methods to get data

### Getting all the addresses

In [37]:
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['addressProvider']['SCORE'])\
    .method("getAllAddresses")\
    .build()
response = icon_service.call(_call)
pprint(response)

{'collateral': {'USDb': 'cx134a1a0309aaf88a00ad1fee4983b7ed43454ab6',
                'sICX': 'cx735ee7e87d30d76a83b27c7bc62a84b5beafa34b'},
 'oTokens': {'oICX': 'cx6dbb85df52d7b12d2c00d673d89c5bf21ee29be2',
             'oUSDb': 'cx2f4c43ec2c69321573e93549d89462de4a25fdbe'},
 'systemContract': {'LendingPool': 'cxbfd0ecb9892ca2e1ae3d0494d48f6618620b7e48',
                    'LendingPoolDataProvider': 'cx726e8bacc442256099dda50438c11d004215afc5'}}


### Getting specific reserve data

In [30]:
#this returns the reserve data of USDb
params = {'_reserve': contracts['sicx']['SCORE']}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider']['SCORE'])\
    .method("getReserveData")\
    .params(params)\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    if values=="0x1" or values =="0x0" or "Address" in key:
        continue
    if key!="lastUpdateTimestamp":
        response[key]=int(values,0)/10**18
    else:
        response[key]=int(values,0)

pprint(response)

{'availableLiquidity': 214.5,
 'baseLTVasCollateral': 0.33,
 'borrowCumulativeIndex': 1.000011493399191,
 'borrowRate': 0.018912999951505187,
 'borrowingEnabled': '0x1',
 'decimals': 1.8e-17,
 'isActive': '0x1',
 'isFreezed': '0x0',
 'lastUpdateTimestamp': 1606843688248792,
 'liquidationBonus': 1e-17,
 'liquidationThreshold': 0.65,
 'liquidityCumulativeIndex': 1.0000022527039965,
 'liquidityRate': 0.002759412089563471,
 'oTokenAddress': 'cxf6e3d64db674c0edbf732462e80e2e3352280548',
 'reserveAddress': 'cx3e9d38175d112cd5cf05a90080cce779f202ba27',
 'totalBorrows': 41.50062730044531,
 'totalLiquidity': 256.00062730044533,
 'usageAsCollateralEnabled': '0x1'}


# Reserve Account Data

In [16]:
#this returns the reserve data of USDb

_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider']['SCORE'])\
    .method("getReserveAccountData")\
    .build()
response = icon_service.call(_call)
# print(response)
for key,values in response.items():
    response[key]=int(values,0)/10**18
        

pprint(response)

{'availableLiquidityBalanceUSD': 24104.87499792432,
 'totalBorrowsBalanceUSD': 900.5229921951135,
 'totalCollateralBalanceUSD': 25005.39799011943,
 'totalLiquidityBalanceUSD': 25005.39799011943}


### Getting data of all reserves

In [17]:
#this returns the reserve data of all the reserves
params = {'_reserve': contracts['sample_token']['SCORE']}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider']['SCORE'])\
    .method("getAllReserveData")\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    for k,v in values.items():
        if v=="0x1" or v =="0x0" or "Address" in k:
            continue
        if k!="lastUpdateTimestamp":
            response[key][k]=int(v,0)/10**18
        else:
            response[key][k]=int(v,0)
pprint(response)

{'USDb': {'availableLiquidity': 23999.99999792432,
          'baseLTVasCollateral': 0.33,
          'borrowCumulativeIndex': 1.0000319286263997,
          'borrowRate': 0.004128273409968422,
          'borrowingEnabled': '0x1',
          'decimals': 1.8e-17,
          'isActive': '0x1',
          'isFreezed': '0x0',
          'lastUpdateTimestamp': 1606289631645781,
          'liquidationBonus': 1e-17,
          'liquidationThreshold': 0.65,
          'liquidityCumulativeIndex': 1.000000664971861,
          'liquidityRate': 0.000131471804680347,
          'oTokenAddress': 'cx4258464e395b3ef3b73a146947eb41393e8c5873',
          'reserveAddress': 'cx19be1be7b4e7750863edf966faac3f42dec21025',
          'totalBorrows': 880.3978710152415,
          'totalLiquidity': 24880.39786893956,
          'usageAsCollateralEnabled': '0x1'},
 'sICX': {'availableLiquidity': 209.75,
          'baseLTVasCollateral': 0.33,
          'borrowCumulativeIndex': 1.0000024672503287,
          'borrowRate': 0.018

### Getting user account data

In [32]:
#returns the user data of all the reserves combined
params = {'_user': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce'}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider']['SCORE'])\
    .method("getUserAccountData")\
    .params(params)\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    if key!="lastUpdateTimestamp":
        response[key]=int(values,0)/10**18
    else:
        response[key]=int(values,0)
pprint(response)

{'currentLiquidationThreshold': 0.65,
 'currentLtv': 0.33,
 'healthFactor': 196.60244264643964,
 'healthFactorBelowThreshold': 0.0,
 'totalBorrowBalanceUSD': 20.250314476059945,
 'totalCollateralBalanceUSD': 6125.0173697722175,
 'totalFeesUSD': 0.0,
 'totalLiquidityBalanceUSD': 6125.0173697722175}


In [49]:
#returns the user data of all the reserves combined
params = {'_reserve': contracts['sample_token']['SCORE'],'_amount':10*10**18,"_fee":1,'_userCurrentBorrowBalanceUSD':0,'_userCurrentFeesUSD':0,'_userCurrentLtv':5*10**17}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider']['SCORE'])\
    .method("calculateCollateralNeededUSD")\
    .params(params)\
    .build()
response = icon_service.call(_call)
response = int(response,0)/10**18
# for key,values in response.items():
#     if key!="lastUpdateTimestamp":
#         response[key]=int(values,0)/10**18
#     else:
#         response[key]=int(values,0)
print(response)

20.0


### Getting user reserve data

In [31]:
#returns the data of a specific user for a specific reserve
#this returns the data of user for USDb reserve
params = {'_user': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce' ,"_reserve":contracts['sicx']['SCORE']}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider']['SCORE'])\
    .method("getUserReserveData")\
    .params(params)\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    if key!="lastUpdateTimestamp":
        response[key]=int(values,0)/10**18
    else:
        response[key]=int(values,0)
pprint(response)

{'borrowRate': 0.018912999951505187,
 'currentBorrowBalance': 40.500628612069235,
 'currentBorrowBalanceUSD': 20.250314306034618,
 'currentOTokenBalance': 250.00056435725696,
 'currentOTokenBalanceUSD': 125.00028217862848,
 'lastUpdateTimestamp': 1606843688248792,
 'liquidityRate': 0.002759412089563471,
 'originationFee': 0.0,
 'principalBorrowBalance': 40.50062730044531,
 'principalBorrowBalanceUSD': 20.250313650222655,
 'useAsCollateral': 1e-18,
 'userBorrowCumulativeIndex': 1.000011493399191}


# Getting All Reserve Data

In [44]:
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider']['SCORE'])\
    .method("getAllReserveData")\
    .build()
response = icon_service.call(_call)
pprint(response)

{'USDb': {'availableLiquidity': '0x3f35b7a91f3815900000',
          'baseLTVasCollateral': '0x853a0d2313c0000',
          'borrowCumulativeIndex': '0xde0b7c160e6fb0d',
          'borrowRate': '0x128fc221d15a6e5',
          'borrowingEnabled': '0x1',
          'decimals': '0x12',
          'isActive': '0x1',
          'isFreezed': '0x0',
          'lastUpdateTimestamp': '0x5b4bfa2ee7bf9',
          'liquidationBonus': '0xa',
          'liquidationThreshold': '0x905438e60010000',
          'liquidityCumulativeIndex': '0xde0b73a80e1fe23',
          'liquidityRate': '0xa78e20466e3ea9',
          'oTokenAddress': 'cx2f4c43ec2c69321573e93549d89462de4a25fdbe',
          'reserveAddress': 'cx134a1a0309aaf88a00ad1fee4983b7ed43454ab6',
          'totalBorrows': '0x6a32602b7ababa00e840',
          'totalLiquidity': '0xa96817d499f2cf90e840',
          'usageAsCollateralEnabled': '0x1'},
 'sICX': {'availableLiquidity': '0x0',
          'baseLTVasCollateral': '0x494654067e10000',
          'borrowCu

### Getting the data of user for all reserves

In [19]:
#returns the data of a specific user for all reserves

params = {'_user': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce'}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider']['SCORE'])\
    .method("getUserAllReserveData")\
    .params(params)\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    for k,v in values.items():
        if k!="lastUpdateTimestamp":
            response[key][k]=int(v,0)/10**18
        else:
            response[key][k]=int(v,0)
pprint(response)

{'USDb': {'borrowRate': 0.004128273409968422,
          'currentBorrowBalance': 0.0,
          'currentBorrowBalanceUSD': 0.0,
          'currentOTokenBalance': 6000.016846751915,
          'currentOTokenBalanceUSD': 6000.016846751915,
          'lastUpdateTimestamp': 0,
          'liquidityRate': 0.000131471804680347,
          'originationFee': 0.0,
          'principalBorrowBalance': 0.0,
          'principalBorrowBalanceUSD': 0.0,
          'useAsCollateral': 1e-18,
          'userBorrowCumulativeIndex': 0.0},
 'sICX': {'borrowRate': 0.01878342822515975,
          'currentBorrowBalance': 40.25027750533986,
          'currentBorrowBalanceUSD': 20.12513875266993,
          'currentOTokenBalance': 250.00024975453397,
          'currentOTokenBalanceUSD': 125.00012487726698,
          'lastUpdateTimestamp': 1606828745337435,
          'liquidityRate': 0.002721732499720759,
          'originationFee': 0.0,
          'principalBorrowBalance': 40.250242359743936,
          'principalBorrow

### Get reserve configurations


In [61]:
#returns the configurations of a specific reserve
params = {"_reserve":contracts['sample_token']['SCORE']}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider']['SCORE'])\
    .method("getReserveConfigurationData")\
    .params(params)\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    response[key]=int(values,0)
pprint(response)


{'baseLTVasCollateral': 330000000000000000,
 'borrowingEnabled': 1,
 'decimals': 18,
 'isActive': 1,
 'liquidationBonus': 10,
 'liquidationThreshold': 650000000000000000,
 'usageAsCollateralEnabled': 1}


### Get all reserves configurations

In [62]:
#returns the configurations of all the   reserves

_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider']['SCORE'])\
    .method("getAllReserveConfigurationData")\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    for k,v in values.items():
        response[key][k]=int(v,0)
pprint(response)


{'USDb': {'baseLTVasCollateral': 330000000000000000,
          'borrowingEnabled': 1,
          'decimals': 18,
          'isActive': 1,
          'liquidationBonus': 10,
          'liquidationThreshold': 650000000000000000,
          'usageAsCollateralEnabled': 1},
 'icx': {'baseLTVasCollateral': 330000000000000000,
         'borrowingEnabled': 1,
         'decimals': 18,
         'isActive': 1,
         'liquidationBonus': 10,
         'liquidationThreshold': 650000000000000000,
         'usageAsCollateralEnabled': 1}}


## Realtime USDb balance of user(with accrued interest)

In [33]:

params = {'_user': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce' }

_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['oToken']['SCORE'])\
    .method("balanceOf")\
    .params(params)\
    .build()
response = icon_service.call(_call)

print(int(response,0))

0


In [20]:

# params = {'_owner': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce' }

_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider']['SCORE'])\
    .method("getOracleAddress")\
    .build()
response = icon_service.call(_call)
print(response)

cx8397f621adac63d786c24cfdf920e683ae320dfd


In [ ]:
995000000570919367819
995000000564440311889
894000001574533095848
894000000615847612243
894000001574533095848
795000001589272371715
795000001731223349511